# Create ping files based on lab choices

## Lab settings

In [1]:

# This dictionnary should be the same as the one used for creating file_defs.
# Here , we quoted only those entries useful for creating ping files
settings={
    'mips_CNRM' : {'AerChemMIP','C4MIP','CFMIP','DAMIP', 'FAFMIP' , 'GeoMIP','GMMIP','ISMIP6',\
                      'LS3MIP','LUMIP','OMIP','PMIP','RFMIP','ScenarioMIP','CORDEX','SIMIP'},
    'mips' : {"CMIP6", "AerChemMIP", "C4MIP", "CFMIP", "DAMIP", "DCPP", "FAFMIP", "GeoMIP", "GMMIP", 
              "HighResMIP", "ISMIP6", "LS3MIP", "LUMIP", "OMIP", "PDRMIP", "PMIP", "RFMIP", "ScenarioMIP", 
              "SolarMIP", "VolMIP", "CORDEX", "DynVar", "SIMIP", "VIACSAB", "SPECS", "CCMI", "CMIP5", 
              "CMIP", "DECK"},
    'max_priority' : 3,
    'tierMax'      : 3,
    "ping_variables_prefix" : "CMIP6_",
    # We account for a file listing the variables which the lab does not want to produce 
    # Format : MIP varname as first column, comment lines begin with '#'
    "excluded_vars_file":"/cnrm/est/USERS/senesi/public/CMIP6/data_request/cnrm/excluded_vars.txt",
    "excluded_vars" : None,
    }

### Read excluded variables list

In [2]:
l=[]
if settings["excluded_vars"] is None and settings["excluded_vars_file"] is not None :
    with open(settings["excluded_vars_file"],'r') as fv : varlines=fv.readlines()
    for line in varlines: 
        fields=line.split()
        if len(fields) > 0 : 
            first=fields[0]
            if first[0] != '#' :  l.append(first)
settings["excluded_vars"]=l
                       

## For getting a comprehensive ping file, one can reset the excluded_var list to None

In [3]:
settings["excluded_vars"]=[]

In [4]:
from dr2xml import select_CMORvars_for_lab, pingFileForRealmsList

01.beta.45


## Select all variables to consider, based on lab settings

In [5]:
svars=select_CMORvars_for_lab(settings, printout=True)


Number of Request Links which apply to MIPS set(['CORDEX', 'GMMIP', 'RFMIP', 'VolMIP', 'CMIP6', 'ScenarioMIP', 'GeoMIP', 'C4MIP', 'PDRMIP', 'CMIP', 'DECK', 'LUMIP', 'CMIP5', 'CFMIP', 'OMIP', 'DAMIP', 'CCMI', 'SolarMIP', 'VIACSAB', 'SIMIP', 'DCPP', 'ISMIP6', 'AerChemMIP', 'PMIP', 'FAFMIP', 'DynVar', 'LS3MIP', 'SPECS', 'HighResMIP'])  is:  341

Number of CMOR variables for these requestLinks is : 1896

Number of CMOR variables once filtered by excluded vars is : 1896

Number of variables with distinct labels is : 1272

Number of simplified vars is : 1896


## Doc for ping files create function

In [6]:
help(pingFileForRealmsList)

Help on function pingFileForRealmsList in module dr2xml:

pingFileForRealmsList(lrealms, svars, dummy='field_atm', dummy_with_shape=False, exact=False, comments=False, prefix='CV_', filename=None)
    Based on a list of realms LREALMS and a list of simplified vars SVARS, create the ping 
    file which name is ~ ping_<realms_list>.xml, which defines fields for all vars in SVARS, 
    with a field_ref which is either 'dummy' or '?<varname>' (depending on logical DUMMY)
    
    If EXACT is True, the match between variable realm string and one of the realm string 
    in the list must be exact. Otherwise, the variable realm must be included in (or include) 
    of the realm list strings
    
    COMMENTS, if not False nor "", will drive the writing of variable description and units 
    as an xml comment. If it is a string, it will be printed before this comment string (and  
    this allows for a line break)
    
    DUMMY, if not false, should be either 'True', for a standard dummy lab

### When using function create_ping_files with argument exact=False, each ping file will adress all variables which realm includes or is included in one of the strings in a realms set  <br><br> e.g for set ['ocean','seaIce'], ping file 'ping_ocean_seaIce.xml' will includes variables which realm is either 'ocean' or 'seaIce' or 'ocean seaIce'

## Create varioius ping files for various sets of realms 

In [7]:
realms=[ 'atmos', 'land','landIce','atmosChem','aerosol']
pingFileForRealmsList(realms,svars,comments=" ",exact=False,dummy=True,
                      filename='ping_atmos_and_co.xml',dummy_with_shape=True)
#pingFileForRealmsList(realms,svars,comments="",exact=False,dummy=True,filename='ping_atmos_nocomments.xml')


838 variables written for ping_atmos_and_co.xml


In [8]:
pingFileForRealmsList(['ocean','seaIce'],svars,comments=" ",exact=False,dummy=True,
                      filename='ping_ocean.xml',dummy_with_shape=True)

274 variables written for ping_ocean.xml


In [9]:
! head -n 5 ping_ocean.xml

<field_definition>
<!-- for variables which realm equals one of _ocean_seaIce-->
   <field id="CV_O18sw"            field_ref="dummy_XYO"        /> <!-- () O18sw : Roche - LSCE -->
   <field id="CV_agessc"           field_ref="dummy_XYO"        /> <!-- (yr) sea_water_age_since_surface_contact : Time elapsed since water was last in surface layer of the ocean. -->
   <field id="CV_areacello"        field_ref="dummy_XY"         /> <!-- (m2) cell_area : Horizontal area of ocean grid cells -->


In [10]:
allrealms=[ ['ocean'], ['seaIce'],['ocnBgchem'], [ 'atmos'], ['land'],['landIce'], ['atmosChem'],[ 'aerosol' ]]

for rs in allrealms :
    pingFileForRealmsList(rs,svars,prefix=settings['ping_variables_prefix'],
                          comments=" ",exact=False, dummy=True,dummy_with_shape=True)


185 variables written for ping_ocean.xml
 97 variables written for ping_seaIce.xml
168 variables written for ping_ocnBgchem.xml
363 variables written for ping_atmos.xml
338 variables written for ping_land.xml
337 variables written for ping_landIce.xml
362 variables written for ping_atmosChem.xml
154 variables written for ping_aerosol.xml
